# PCA for Source Separation of Ventilation and Cardiovascular Activity in Electrical Impedance Tomography (EIT)

## Introduction
In this exercise we use PCA for processing image sequences of thoracic electrical impedance tomography (EIT) signals. EIT is a non-invasive, radiation-free imaging modality which uses small alternating currents to measure bioimpedance of the thorax [1]. These measurements are then converted into image sequences of thoracic impedance changes representing ventilation (i.e., air exchange in the lungs) and cardiovascular activity (e.g., heart movement or blood volume changes in heart and lungs). 

In order to analyze these data it is important to properly separate ventilation and cardiovascular activity. Besides common techniques such as frequency filtering or ECG-triggered averaging, PCA can be used for separating these two sources of signals.
The present example uses the method proposed by Deibele et al. [2] for which the block diagram is shown below:

<img src="FlowChart_Deibele.png">


## References

[1] I. Frerichs et al., “Chest electrical impedance tomography examination, data analysis, terminology, clinical use and recommendations: consensus statement of the TRanslational EIT developmeNt stuDy group,” Thorax, vol. 72, no. 1, pp. 83–93, Jan. 2017, doi: [10.1136/thoraxjnl-2016-208357](https://dx.doi.org/10.1136/thoraxjnl-2016-208357).

[2] J. M. Deibele, H. Luepschen, and S. Leonhardt, “Dynamic separation of pulmonary and cardiac changes in electrical impedance tomography,” Physiological Measurement, vol. 29, no. 6, pp. S1–S14, Jun. 2008, doi: [10.1088/0967-3334/29/6/S01](https://dx.doi.org/10.1088/0967-3334/29/6/S01).

In [1]:
import numpy as np
from scipy.io import loadmat
from scipy.signal import filtfilt, butter
from scipy.linalg import eigh

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)  # initiate notebook for offline plot

from ecgdetectors import Detectors

In [3]:
# load data
data = loadmat('EIT_Data.mat')
t = data['tEit'].flatten()[data['IdxRange'].flatten().astype(bool)]
fs = 1/np.median(np.diff(t))
imgs_eit = data['Imgs']
b, a = butter(4, np.asarray([0.1, 12]), fs=fs, btype='bandpass')
imgs_eit = filtfilt(b, a, imgs_eit, axis=-1)
imgs_eit *= 1E3  # adapt scaling for plotting

# ECG data to be used for bonus question
ecg = {'time': data['Ecg'][0][0][2], 'value': data['Ecg'][0][0][1], 'fs': data['Ecg'][0][0][3]}
ecg_range = (ecg['time'] > t[0]) & (ecg['time'] < t[-1])
ecg['time'] = ecg['time'][ecg_range]
ecg['value'] = ecg['value'][ecg_range]

# force all timings to start at zero
t -= t[0]
ecg['time'] -= ecg['time'][0]

In [4]:
# plot input data
fig = make_subplots(rows=1, cols=2, column_widths=[1, 2])
fig.update_layout(width=950, height=400)

fig.add_trace(go.Heatmap(z=np.std(imgs_eit, axis=-1), zmin=0, 
                         showscale=False, colorscale='magma'), row=1, col=1)
fig.update_yaxes(title='Right', showticklabels=False, autorange="reversed", row=1, col=1)
fig.update_xaxes(title='Dorsal', showticklabels=False, row=1, col=1)
fig.update_layout(title='Overall EIT Activity')

fig.add_trace(go.Scatter(x=t, y=np.nansum(imgs_eit, axis=(0, 1)), 
                         name='Overall Sum Signal', line_color='black'), row=1, col=2)
fig.update_xaxes(title='Time (s)', row=1, col=2)
fig.update_yaxes(title='Impedance Change \\Delta Z (A.U.)', row=1, col=2)


In [5]:
def compute_principal_components(X):
    # perform PCA and compute principal components (pc) and eigenvalues of X
    A = np.dot(X.transpose(), X)  # covariance matrix
    [eigenvalues, eigenvectors] = eigh(A)
    pc = np.dot(X, eigenvectors)
    return pc, eigenvalues

def estimate_cardiac_frequency(s, fs):
    sign_cardiac = np.sign(s)
    # find where zero crossings occured
    index = np.where(np.diff(sign_cardiac) == 2)[0]
    # interpolate to a sub-sample resolution
    pos = index + s[index + 1] / (s[index + 1] - s[index])
    # interpolate RR interval values
    rr = np.diff(pos) / fs
    return 1/np.median(rr)

def lms(A, B):
    if B.ndim == 1:
        B = np.asmatrix(B).transpose()
    tmp = np.linalg.solve(np.dot(B.transpose(), B), B.transpose())
    return np.dot(np.dot(B, tmp), A)

# separate ventilation and cardiovascular activity using PCA
# according to the algorithm by Deibele et al., PhysMeas, 2008
# https://dx.doi.org/10.1088/0967-3334/29/6/S01
imgs_tmp = np.reshape(imgs_eit, [-1, imgs_eit.shape[-1]])
are_valid_pixels = np.all(~np.isnan(imgs_tmp), 1)
X = imgs_tmp[are_valid_pixels, :].transpose()

# first approximation (see block diagram)
X = X - np.repeat(np.reshape(np.mean(X, 0), [1, -1]), X.shape[0], 0)
PC1, lambda1 = compute_principal_components(X)
Bv = PC1[:, -1]
Xv_ = lms(X, Bv)
Xc_ = X - Xv_

# second approximation (see block diagram)
Xc_ = Xc_ - np.repeat(np.reshape(np.mean(Xc_, 0), [1, -1]), Xc_.shape[0], 0)
b, a = butter(6, np.asarray([0.92, 4.6]), fs=fs, btype='bandpass')
Xc_bp = filtfilt(b, a, Xc_, axis=0)
PC2, lambda2 = compute_principal_components(Xc_bp)
Bc_ = PC2[:, -2:]
fc = estimate_cardiac_frequency(Bc_[:, 0], fs)

# create cardiac template functions
Bc = np.hstack((Bc_, np.roll(Bc_, int(fs/fc/3), 0),
                np.roll(Bc_, -int(fs/fc/3), 0)))
Xc1 = lms(Xc_, Bc)
Xc2 = lms(Xv_, Bc)
Xc = (Xc1 + Xc2).transpose()
Xv = (Xv_ - Xc2).transpose()

# cardiovascular activity
imgs_card = np.full(imgs_tmp.shape, np.nan)
imgs_card[are_valid_pixels, :] = Xc
imgs_card = imgs_card.reshape(imgs_eit.shape)
# ventilation activity
imgs_vent = np.full(imgs_tmp.shape, np.nan)
imgs_vent[are_valid_pixels, :] = Xv
imgs_vent = imgs_vent.reshape(imgs_eit.shape)

In [6]:
# plot ventilation acticity
fig = make_subplots(rows=1, cols=2, column_widths=[1, 2])
fig.update_layout(width=950, height=400)

fig.add_trace(go.Heatmap(z=np.std(imgs_vent, axis=-1), zmin=0, 
                         showscale=False, colorscale='magma'), row=1, col=1)
fig.update_yaxes(title='Right', showticklabels=False, autorange="reversed", row=1, col=1)
fig.update_xaxes(title='Dorsal', showticklabels=False, row=1, col=1)
fig.update_layout(title='Ventilation Activity')

fig.add_trace(go.Scatter(x=t, y=np.nanmean(imgs_vent, axis=(0, 1)), 
                         name='Mean Signal', line_color='black'), row=1, col=2)
fig.update_xaxes(title='Time (s)', row=1, col=2)
fig.update_yaxes(title='Impedance Change \\Delta Z (A.U.)', row=1, col=2)

# add example signals in ??? regions
regions = {'Region RL': ([10, 14], 'magenta'), 'Region LL': ([22, 14], 'orange')}
for reg, tmp in regions.items():
    fig.add_scatter(x=[tmp[0][0]], y=[tmp[0][1]], mode='markers', marker_symbol='square-open', 
                    marker_size=10, legendgroup=reg, marker_color=tmp[1], name=reg, row=1, col=1)
    fig.add_trace(go.Scatter(x=t, y=imgs_vent[tmp[0][1], tmp[0][0], :], legendgroup=reg, 
                             showlegend=False, name=reg, line_color=tmp[1]), row=1, col=2) 
    
fig.show()

In [7]:
# plot cardiovascular acticity
fig = make_subplots(rows=1, cols=2, column_widths=[1, 2])
fig.update_layout(width=950, height=400)

fig.add_trace(go.Heatmap(z=np.std(imgs_card, axis=-1), zmin=0, 
                         showscale=False, colorscale='magma'), row=1, col=1)
fig.update_yaxes(title='Right', showticklabels=False, autorange="reversed", row=1, col=1)
fig.update_xaxes(title='Dorsal', showticklabels=False, row=1, col=1)
fig.update_layout(title='Cardiovascular Activity')

fig.add_trace(go.Scatter(x=t, y=np.nanmean(imgs_card, axis=(0, 1)), 
                         name='Mean Signal', line_color='black'), row=1, col=2)
fig.update_xaxes(title='Time (s)', row=1, col=2)
fig.update_yaxes(title='Impedance Change \\Delta Z (A.U.)', row=1, col=2)

# add three example signals in ??? regions
regions = {'Region A': ([18, 9], 'green'), 'Region B': ([10, 15], 'blue'), 'Region C': ([23, 15], 'red')}
for reg, tmp in regions.items():
    fig.add_scatter(x=[tmp[0][0]], y=[tmp[0][1]], mode='markers', marker_symbol='square-open', 
                    marker_size=10, legendgroup=reg, marker_color=tmp[1], name=reg, row=1, col=1)
    fig.add_trace(go.Scatter(x=t, y=imgs_card[tmp[0][1], tmp[0][0], :], legendgroup=reg, 
                             showlegend=False, name=reg, line_color=tmp[1]), row=1, col=2)    
fig.show()

# Exercise Questions
Please provide your answers directly below each question.

---

## Question 1 
Determine the frequency of the ventilation activity (i.e., the respiratory rate), both expressed in `Hz` and `respirations/min`.

In [8]:
vent_signal = np.nanmean(imgs_vent, axis=(0,1))
vent_freq = estimate_cardiac_frequency(vent_signal, fs)
print("Ventilation frequency (Hz):", vent_freq)
print("Ventilation rate (resp/min):", vent_freq*60)

Ventilation frequency (Hz): 0.32336472979853587
Ventilation rate (resp/min): 19.40188378791215


**Answer:** The estimated frequency of the ventilation activity is 0.323 Hz, which corresponds to approximately 19.4 respirations per minute.

## Question 2
Determine the frequency of the cardiovascular activity, both expressed in `Hz` and `beats/min`.

In [35]:
card_signal = np.nanmean(imgs_card, axis=(0,1))
card_freq = estimate_cardiac_frequency(card_signal, fs)
print("Cardiovascular frequency (Hz):", card_freq)
print("Heart rate (beats/min):", card_freq*60)

Cardiovascular frequency (Hz): 0.975007622106122
Heart rate (beats/min): 58.50045732636732


**Answer:** The estimated frequency of the cardiovascular activity is 0.975Hz, which corresponds to approximately 58.5 BPM.

## Question 3
Determine the follwing three values:

- i) the maximal amplitude of ventilation activity; 
- ii) the maximal amplitude of cardiovascular activity; and 
- iii) the ratio between i) and ii), i.e., ventilation vs cardiovascular activity. 

In [36]:
vent_amp = np.nanmax(np.abs(imgs_vent))
card_amp = np.nanmax(np.abs(imgs_card))
ratio = vent_amp / card_amp
print("Ventilation max amplitude:", vent_amp)
print("Cardiovascular max amplitude:", card_amp)
print("Ventilation/Cardiovascular ratio:", ratio)

Ventilation max amplitude: 0.053047195655208094
Cardiovascular max amplitude: 0.006880552621845422
Ventilation/Cardiovascular ratio: 7.709728937583561


**Question:** The maximal amplitude of ventilation activity is 0.0530, while the maximum amplitude of cardiovascular activity is 0.00688. This yields a ventilation to cardiac ratio of 7.71.

## Question 4
Determine the eigenvalues of the first three principal components resulting from the first PCA (see variable `PC1`).

In [10]:
print("First 3 eigenvalues:", lambda1[-3:])

First 3 eigenvalues: [  0.80315244   2.11148888 107.58688656]


**Answer:** The three largest eigenvalues of the PCA decomposition are 0.0803, 2.11, and 107.6. Since eigh() returns eigenvalues in ascending order, the largest values are the last ones in lambda1 (hence the lambda1[-3].)

## Question 5
Similar to Question 4, determine the two most dominant frequencies for the first three principal components. <i>Note that this would lead to a total of 6 values, 2 frequencies for each of the 3 PCA components. However, for some components only one dominant frequency might be present.</i><br>
For all 3 PCA components, which one is rather related to ventilation or cardiovascular activity?

In [12]:
from scipy.signal import find_peaks

pc_components = PC1[:, -3:]
n_peaks=2

for i in range(3):
    n = len(pc_components[:, i])
    fft_vals = np.fft.rfft(pc_components[:, i])
    fft_freq = np.fft.rfftfreq(n, 1/fs)
    fft_mag = np.abs(fft_vals)
    
    peaks, _ = find_peaks(fft_mag, height=np.max(fft_mag)*0.05)
    peak_freqs = fft_freq[peaks]
    peak_mags = fft_mag[peaks]
    
    sorted_indices = np.argsort(peak_mags)[::-1]
    dominant_freqs = peak_freqs[sorted_indices][:n_peaks]

    print(f"PC{i+1} dominant frequencies (Hz):", np.sort(dominant_freqs))

PC1 dominant frequencies (Hz): [0.31954524 0.95863573]
PC2 dominant frequencies (Hz): [0.31954524 0.95863573]
PC3 dominant frequencies (Hz): [0.31954524 0.39943156]


**Answer:** In both PC1 and PC2, the strongest peaks occur at 0.32Hz, which corresponds to ventilation, and 0.96Hz, which corresponds to cardiac activity. As for PC3, its strongest peaks occur at 0.32Hz and 0.40Hz, which correspond respectively to ventilation activity and a weaker secondary harmonic.

## Question 6
The three example signals (Regions A to C: **green, blue, red**) of cardiovascular activity show the impedance change over time. 
Can you guess the underlying anatomical structure for each of the three example signals (Regions A to C: **green, blue, red**).

**Answer:** Region A (green) has the largest amplitude and sharpest pulsations and is located somewhat centrally, which indicates that it likely corresponds to the heart area, where the pulse is the strongest.

Region B (blue) has the smallest and most attenuated pulse waveform and is located in a more peripheral area in the image. This could correspond to areas of the lungs, where the heartbeat signal becomes weaker.

Region C (red) shows intermediate amplitude that is slightly larger than the blue one, which suggests that it comes from a border region near the heart (therefore in between the heart and the lungs).


## Question 7 - *Bonus Question* (not graded) 
Can you detect the QRS peaks (e.g., using [`qrs = Detectors(fs).engzee_detector(ecg)`](https://github.com/berndporr/py-ecg-detectors/blob/7a1ca569190a4c17885089c97480efd114c64120/ecgdetectors.py#L277)) on the ECG signal (see variable `ecg`) and plot them together with the cardiovascular EIT activity?